
# **Laboratorio de Implementacion III**: **Forecasting para empresa de consumo masivo**

##*   **NOTEBOOK 3**: Experimento AutoGluon

## **Integrantes**:  
*   Lucia Martinez


La logica de este notebook es la siguiente


*  con el dataframe generado luego del feature engineering, se hace el AutoGluon

















In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 3.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of openxlab to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
   ━━━━━━━━━━━━━━━

In [ ]:
import pandas as pd
from pathlib import Path
import gc
import warnings
from	pandas	import	Series
import numpy as np
import warnings
warnings.filterwarnings("ignore")
from autogluon.timeseries import TimeSeriesDataFrame, TimeSeriesPredictor




In [ ]:
gc.collect()
filepath1 = Path('/content/drive/MyDrive/Labo3/df_monthly_full.txt.gz')
df_monthly_full = pd.read_csv(filepath1, compression='gzip', sep='\t')
df_monthly_full.head(5)

,product_id,periodo,tn,lag1m,delta1,lag2m,delta2,lag3m,delta3,lag4m,...,lag11m,delta11,lag12m,delta12,monthly_percentage,tn_roll_mean_3m,tn_roll_mean_6m,tn_ewma_alpha02,tn_ewma_alpha05,tn_ewma_alpha08
0,20001,2017-01-01,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,934.772220,934.772220,934.772220
1,20001,2017-02-01,798.01620,934.77222,-136.75602,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,-0.146299,934.772220,934.772220,907.421016,866.394210,825.367404
2,20001,2017-03-01,1303.35771,798.01620,505.34151,934.77222,368.58549,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,0.633247,866.394210,866.394210,986.608355,1084.875960,1207.759649
3,20001,2017-04-01,1069.96130,1303.35771,-233.39641,798.01620,271.94510,934.77222,135.18908,NaN,...,NaN,NaN,NaN,NaN,-0.179073,1012.048710,1012.048710,1003.278944,1077.418630,1097.520970
4,20001,2017-05-01,1502.20132,1069.96130,432.24002,1303.35771,198.84361,798.01620,704.18512,934.77222,...,NaN,NaN,NaN,NaN,0.403977,1057.111737,1026.526858,1103.063419,1289.809975,1421.265250


In [ ]:
# dataframe estaticos de variables categoricas
filepath5 = Path('/content/drive/MyDrive/Labo3/dataset_grupos.txt.gz')
df_grupo = pd.read_csv(filepath5, compression='gzip', sep='\t')

# Seleccionar columnas necesarias para el DataFrame de features estáticos
cols_estaticas = ['product_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']
df_static = df_grupo[cols_estaticas].drop_duplicates('product_id').set_index('product_id')


# Convertir a categórico
for col in df_static.columns:
    df_static[col] = df_static[col].astype('category')


df_static.head()


,cat1,cat2,cat3,brand,sku_size
product_id,,,,,
20524,HC,VAJILLA,Cristalino,Importado,500.0
20311,HC,VAJILLA,Cristalino,Importado,1000.0
20654,PC,DEOS,Aero,DEOS2,60.0
21005,FOODS,SOPAS Y CALDOS,Caldos Granulados,MAGGI,12.0
20974,FOODS,SOPAS Y CALDOS,Caldos Granulados,MAGGI,12.0


In [ ]:
df1 = pd.merge(df_monthly_full, df_static, how='left', left_on='product_id', right_on='item_id')
df1 = df1.drop(columns=['item_id'])
df1.head()



,product_id,periodo,tn,lag1m,delta1,lag2m,delta2,lag3m,delta3,lag4m,...,tn_roll_mean_3m,tn_roll_mean_6m,tn_ewma_alpha02,tn_ewma_alpha05,tn_ewma_alpha08,cat1,cat2,cat3,brand,sku_size
0,20001,2017-01-01,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,934.772220,934.772220,934.772220,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
1,20001,2017-02-01,798.01620,934.77222,-136.75602,NaN,NaN,NaN,NaN,NaN,...,934.772220,934.772220,907.421016,866.394210,825.367404,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
2,20001,2017-03-01,1303.35771,798.01620,505.34151,934.77222,368.58549,NaN,NaN,NaN,...,866.394210,866.394210,986.608355,1084.875960,1207.759649,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
3,20001,2017-04-01,1069.96130,1303.35771,-233.39641,798.01620,271.94510,934.77222,135.18908,NaN,...,1012.048710,1012.048710,1003.278944,1077.418630,1097.520970,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
4,20001,2017-05-01,1502.20132,1069.96130,432.24002,1303.35771,198.84361,798.01620,704.18512,934.77222,...,1057.111737,1026.526858,1103.063419,1289.809975,1421.265250,HC,ROPA LAVADO,Liquido,ARIEL,3000.0


In [ ]:
df1 = df1.rename(columns={
    'product_id': 'item_id',
    'periodo': 'timestamp'
})

print(df1.columns.tolist())


['item_id', 'timestamp', 'tn', 'lag1m', 'delta1', 'lag2m', 'delta2', 'lag3m', 'delta3', 'lag4m', 'delta4', 'lag5m', 'delta5', 'lag6m', 'delta6', 'lag7m', 'delta7', 'lag8m', 'delta8', 'lag9m', 'delta9', 'lag10m', 'delta10', 'lag11m', 'delta11', 'lag12m', 'delta12', 'monthly_percentage', 'tn_roll_mean_3m', 'tn_roll_mean_6m', 'tn_ewma_alpha02', 'tn_ewma_alpha05', 'tn_ewma_alpha08', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']


In [ ]:
df1.head()

,item_id,timestamp,tn,lag1m,delta1,lag2m,delta2,lag3m,delta3,lag4m,...,tn_roll_mean_3m,tn_roll_mean_6m,tn_ewma_alpha02,tn_ewma_alpha05,tn_ewma_alpha08,cat1,cat2,cat3,brand,sku_size
0,20001,2017-01-01,934.77222,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,934.772220,934.772220,934.772220,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
1,20001,2017-02-01,798.01620,934.77222,-136.75602,NaN,NaN,NaN,NaN,NaN,...,934.772220,934.772220,907.421016,866.394210,825.367404,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
2,20001,2017-03-01,1303.35771,798.01620,505.34151,934.77222,368.58549,NaN,NaN,NaN,...,866.394210,866.394210,986.608355,1084.875960,1207.759649,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
3,20001,2017-04-01,1069.96130,1303.35771,-233.39641,798.01620,271.94510,934.77222,135.18908,NaN,...,1012.048710,1012.048710,1003.278944,1077.418630,1097.520970,HC,ROPA LAVADO,Liquido,ARIEL,3000.0
4,20001,2017-05-01,1502.20132,1069.96130,432.24002,1303.35771,198.84361,798.01620,704.18512,934.77222,...,1057.111737,1026.526858,1103.063419,1289.809975,1421.265250,HC,ROPA LAVADO,Liquido,ARIEL,3000.0


In [ ]:
# En este modelo,no se consideran covariables futuras.
# Un proximo modelo podria ser con las variables categóricas que si se conocen para el futuro
from autogluon.timeseries import TimeSeriesPredictor
from pathlib import Path
from datetime import datetime

now = datetime.now().strftime("%Y%m%d_%H%M%S")
filepath2 = Path(f'/content/drive/MyDrive/Labo3/predictor_{now}')

# NO definir covariates si no tienes covariables futuras
predictor = TimeSeriesPredictor(
    target='tn',
    prediction_length=2,
    freq='M',
    eval_metric='WQL',
    # known_covariates_names=covariates,  <-- eliminar o comentar esta línea
    path=str(filepath2)
).fit(
    df1,
    num_val_windows=3,
    val_step_size=1
)




Frequency 'M' stored as 'ME'
Beginning AutoGluon training...
AutoGluon will save models to '/content/drive/MyDrive/Labo3/predictor_20250614_154313'
=================== System Info ===================
AutoGluon Version:  1.3.1
Python Version:     3.11.13
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Mar 30 16:01:29 UTC 2025
CPU Count:          2
GPU Count:          0
Memory Avail:       10.20 GB / 12.67 GB (80.5%)
Disk Space Avail:   14.13 GB / 15.00 GB (94.2%)

Fitting with arguments:
{'enable_ensemble': True,
 'eval_metric': WQL,
 'freq': 'ME',
 'hyperparameters': 'default',
 'known_covariates_names': [],
 'num_val_windows': 3,
 'prediction_length': 2,
 'quantile_levels': [0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9],
 'random_seed': 123,
 'refit_every_n_windows': 1,
 'refit_full': False,
 'skip_model_selection': False,
 'target': 'tn',
 'val_step_size': 1,
 'verbosity': 2}

train_data with frequency 'MS' has been resampled to freq

config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/821M [00:00<?, ?B/s]

	-0.2118       = Validation score (-WQL)
	81.50   s     = Training runtime
	31.58   s     = Validation (prediction) runtime
Training timeseries model ChronosFineTuned[bolt_small]. 
	Skipping covariate_regressor since the dataset contains no covariates or static features.


config.json:   0%|          | 0.00/1.12k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/191M [00:00<?, ?B/s]

	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	Saving fine-tuned model to /content/drive/MyDrive/Labo3/predictor_20250614_154313/models/ChronosFineTuned[bolt_small]/W0/fine-tuned-ckpt
	Skipping covariate_regressor since the dataset contains no covariates or static features.
	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	Saving fine-tuned model to /content/drive/MyDrive/Labo3/predictor_20250614_154313/models/ChronosFineTuned[bolt_small]/W1/fine-tuned-ckpt
	Skipping covariate_regressor since the dataset contains no covariates or static features.
	Fine-tuning on the CPU detected. We recommend using a GPU for faster fine-tuning of Chronos.
	Saving fine-tuned model to /content/drive/MyDrive/Labo3/predictor_20250614_154313/models/ChronosFineTuned[bolt_small]/W2/fine-tuned-ckpt
	-0.2116       = Validation score (-WQL)
	3409.08 s     = Training runtime
	5.66    s     = Validation (prediction) runtim

In [ ]:
predictor.leaderboard()

,model,score_val,pred_time_val,fit_time_marginal,fit_order
0,WeightedEnsemble,-0.191758,95.164219,7.940067,13
1,TemporalFusionTransformer,-0.204168,2.113873,1381.239779,9
2,DeepAR,-0.210619,3.077556,379.262266,10
3,ChronosFineTuned[bolt_small],-0.211602,5.662819,3409.082877,8
4,ChronosZeroShot[bolt_base],-0.211830,31.579255,81.495214,7
5,PatchTST,-0.215837,0.676769,184.111785,11
6,DynamicOptimizedTheta,-0.222276,6.764527,16.975928,5
7,AutoETS,-0.227737,53.940097,106.696035,6
8,TiDE,-0.231237,1.571431,964.936375,12
9,SeasonalNaive,-0.266446,3.329470,6.840854,1


In [ ]:

predictor.feature_importance()

Computing feature importance


,importance,stdev,n,p99_low,p99_high
cat1,0.000072,0.001233,5.0,-0.002466,0.002610
cat2,0.000131,0.000643,5.0,-0.001193,0.001454
cat3,0.000947,0.000694,5.0,-0.000482,0.002376
brand,-0.000673,0.001028,5.0,-0.002790,0.001443
sku_size,-0.000041,0.000643,5.0,-0.001365,0.001283
lag1m,0.000361,0.001614,5.0,-0.002963,0.003685
delta1,0.001346,0.001741,5.0,-0.002239,0.004931
lag2m,-0.000281,0.001300,5.0,-0.002958,0.002397
delta2,-0.000210,0.002451,5.0,-0.005256,0.004837
lag3m,-0.000725,0.000837,5.0,-0.002450,0.000999


In [ ]:
import pandas as pd

# Última fecha en df1
last_date = pd.to_datetime(df1['timestamp']).max()

# Crear fechas futuras (por ejemplo, los próximos 2 meses)
future_dates = pd.date_range(start=last_date + pd.offsets.MonthBegin(1), periods=2, freq='M')

# Lista de productos únicos
products = df1['item_id'].unique()

# Crear dataframe con combinaciones item_id x fechas futuras
future_df = pd.DataFrame([
    {'item_id': pid, 'timestamp': date}
    for pid in products
    for date in future_dates
])

future_df['timestamp'] = future_df['timestamp'].dt.strftime('%Y-%m-%d')  # si se necesita en string

# Asegurarse que future_df no tiene la columna target (tn) ni otras

future_df.head()


,item_id,timestamp
0,20001,2020-01-31
1,20001,2020-02-29
2,20002,2020-01-31
3,20002,2020-02-29
4,20003,2020-01-31


In [ ]:
# Covariables estáticas únicas por producto
static_covs = df1[['item_id', 'cat1', 'cat2', 'cat3', 'brand', 'sku_size']].drop_duplicates()

# Hacer merge para añadir covariables categóricas
known_covariates_future = future_df.merge(static_covs, on='item_id', how='left')

# Definir las covariables continuas que usa el modelo
cont_cols = ['lag1m', 'delta1', 'lag2m', 'delta2', 'lag3m', 'delta3', 'lag4m', 'delta4',
             'lag5m', 'delta5', 'lag6m', 'delta6', 'lag7m', 'delta7', 'lag8m', 'delta8',
             'lag9m', 'delta9', 'lag10m', 'delta10', 'lag11m', 'delta11', 'lag12m', 'delta12',
             'monthly_percentage', 'tn_roll_mean_3m', 'tn_roll_mean_6m',
             'tn_ewma_alpha02', 'tn_ewma_alpha05', 'tn_ewma_alpha08']

# Asignar cero a las covariables continuas (o el promedio histórico si prefieres)
for col in cont_cols:
    known_covariates_future[col] = 0

# Finalmente seleccionar solo las columnas de covariables para pasar al modelo
covariate_cols = ['cat1', 'cat2', 'cat3', 'brand', 'sku_size'] + cont_cols
known_covariates_future_final = known_covariates_future[covariate_cols]

known_covariates_future_final.head()





,cat1,cat2,cat3,brand,sku_size,lag1m,delta1,lag2m,delta2,lag3m,...,lag11m,delta11,lag12m,delta12,monthly_percentage,tn_roll_mean_3m,tn_roll_mean_6m,tn_ewma_alpha02,tn_ewma_alpha05,tn_ewma_alpha08
0,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,HC,ROPA LAVADO,Liquido,ARIEL,3000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,HC,ROPA LAVADO,Liquido,LIMPIEX,3000.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,FOODS,ADEREZOS,Mayonesa,NATURA,475.0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
predictions = predictor.predict(df1,
                                known_covariates = known_covariates_future)

data with frequency 'MS' has been resampled to frequency 'ME'.
Model not specified in predict, will default to the model with the best validation score: WeightedEnsemble


In [ ]:
# Copiar predicciones para no alterar el original
predictions_v1 = predictions.copy()

# Resetear índice para tener columnas planas
predictions_v1.reset_index(inplace=True)

# Filtrar solo las columnas necesarias
predictions_v1 = predictions_v1[["item_id", "timestamp", "mean"]]

# Filtrar por la fecha deseada
fecha_objetivo = "2020-02-29"
predictions_v1 = predictions_v1[predictions_v1["timestamp"] == fecha_objetivo]

# Renombrar columnas y eliminar la columna 'timestamp'
predictions_v1 = predictions_v1.rename(columns={"item_id": "product_id", "mean": "tn"}).drop(columns=["timestamp"])

# Mostrar primeras filas
predictions_v1.head(20)


,product_id,tn
1,20001,1281.969362
3,20002,1055.756197
5,20003,731.955959
7,20004,520.956146
9,20005,492.266031
11,20006,428.818414
13,20007,389.497323
15,20008,378.511297
17,20009,478.386281
19,20010,379.778421


In [ ]:
filepath3 = Path('/content/drive/MyDrive/Labo3/predictionv1.txt.gz')

predictions_v1.to_csv(filepath3, index=False, compression='gzip', sep='\t')


In [ ]:
# Definir lista objetivo a predecir
filepath4 = Path('/content/drive/MyDrive/Labo3/product_id_apredecir201912.txt')
df_objetivo = pd.read_csv(filepath4, header=None, names=['product_id'])
df_objetivo = df_objetivo[df_objetivo['product_id'] != 'product_id']  # elimina fila de encabezado duplicado
df_objetivo['product_id'] = pd.to_numeric(df_objetivo['product_id'], errors='coerce')
df_objetivo = df_objetivo.dropna(subset=['product_id'])
df_objetivo['product_id'] = df_objetivo['product_id'].astype(int)

In [ ]:
# Asegurar mismo tipo de dato (opcional pero recomendado)
predictions_v1['product_id'] = predictions_v1['product_id'].astype(str)
df_objetivo['product_id'] = df_objetivo['product_id'].astype(str)

# Filtrar solo productos deseados
dka = predictions_v1[predictions_v1['product_id'].isin(df_objetivo['product_id'])]

# Ver ejemplo
print(dka.head())


# Guardar los archivos filtrados en CSV (puedes cambiar la extensión y separador si quieres)
output_folder = Path('/content/drive/MyDrive/Labo3/')
dka.to_csv(output_folder / 'dfAutogluon.csv', index=False)

print("Archivos filtrados guardados correctamente.")

  product_id           tn
1      20001  1281.969362
3      20002  1055.756197
5      20003   731.955959
7      20004   520.956146
9      20005   492.266031
Archivos filtrados guardados correctamente.
